# 📊 Evaluación rigurosa de modelos RNN: Perplejidad y palabras fuera de vocabulario

Este notebook presenta una implementación estructurada para evaluar modelos LSTM en tareas de modelado de lenguaje, 
haciendo especial énfasis en métricas como la *Perplejidad* (PP) y el manejo de palabras fuera de vocabulario (*Out-Of-Vocabulary*, OOV).

## 🔧 BLOQUE 1: Setup y descarga de datos

- Instalación de dependencias necesarias (`torch`, `torchtext`, `nltk`, etc.).
- Descarga del dataset **WikiText-2**.
- Tokenización del corpus.
- Construcción de vocabulario con diferentes tamaños:
  - 10,000 palabras
  - 30,000 palabras
  - 50,000 palabras

# 🔧 Bloque 1: Setup y descarga de datos

### 📚 Teoría
El dataset **WikiText-2** es un corpus ampliamente utilizado para tareas de modelado de lenguaje. Contiene texto derivado de artículos de Wikipedia y es útil para entrenar y evaluar modelos de predicción de texto.

Antes de entrenar cualquier modelo, es importante:
1. Descargar el corpus.
2. Tokenizar el texto.
3. Construir un vocabulario de tamaño controlado (ej. 10^4, 3x10^4, 5x10^4 tokens).
4. Dividir en datasets de entrenamiento, validación y prueba.

Utilizaremos `torchtext`, que facilita todo este proceso con utilidades listas para usar.




In [2]:

# Instalación de dependencias necesarias (si no están disponibles)
!pip install torch torchtext


   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 4.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 3.7 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

# Selección de tokenizer tipo 'basic_english'
tokenizer = get_tokenizer("basic_english")

# Carga de datasets
train_iter = WikiText2(split='train')
valid_iter = WikiText2(split='valid')
test_iter = WikiText2(split='test')

# Tokenización y conteo de frecuencias
counter = Counter()
for line in train_iter:
    counter.update(tokenizer(line))

# Ejemplo: Crear vocabulario de tamaño 10^4
vocab_size = 10_000
vocab = Vocab(counter, max_size=vocab_size, specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

c:\Users\Marcelo\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchtext\__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: [WinError 127] The specified procedure could not be found

## 🧠 BLOQUE 2: Definición del modelo

Se define un modelo LSTM con las siguientes características:

- Embeddings de dimensión 300.
- Dos capas LSTM con tamaño oculto de 512.
- Capa final densa con softmax para predecir la siguiente palabra.

**Componentes del modelo:**
- `Embedding layer`
- `LSTM layers (stacked)`
- `Linear output layer`

## 🏋️ BLOQUE 3: Entrenamiento

Entrenamos tres versiones del modelo, una por cada tamaño de vocabulario.

**Funciones incluidas:**
- Entrenamiento y validación por épocas.
- Registro de métricas:
  - *Perplejidad* sobre conjunto de validación.
  - Conteo y porcentaje de palabras OOV.

## 🧩 BLOQUE 4: Manejo de OOV

Se comparan tres estrategias de tratamiento para palabras fuera de vocabulario:

1. **Token `<UNK>`:** se reemplazan todas las palabras desconocidas por un token especial.
2. **Modelo char-level:** backoff a un modelo de caracteres para predecir embeddings.
3. **Similitud de Levenshtein:** reemplazo por la palabra más cercana en el vocabulario.

## 📈 BLOQUE 5: Evaluación

Métricas medidas:

- **Perplejidad** en el conjunto de prueba.
- **Porcentaje de OOV** en test.
- Comparación entre modelos con distintos vocabularios y estrategias OOV.

## 📊 BLOQUE 6: Análisis y visualización

Incluye:

- Gráficos de perplejidad vs. tamaño de vocabulario.
- Cobertura del vocabulario vs. porcentaje de OOV.
- Comparativa de estrategias de manejo de OOV.

## 📘 BLOQUE 7: Reporte final

Resumen de hallazgos:

- Impacto del tamaño del vocabulario sobre la perplejidad.
- Eficacia relativa de cada estrategia de OOV.
- Recomendaciones para aplicaciones en producción.

**Exportable a PDF o documento académico.**